In [1]:
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
import numpy as np
import pandas as pd
import pickle

In [2]:
def GetTodayOption(year, month, day):
    keys = {'syear': year, 'smonth': month, 'sday': day} # 要傳給期貨交易所的 key
    r = requests.get("http://www.taifex.com.tw/chinese/3/3_2_1.asp", data = keys) # 抓取這個 key 回傳網頁的原始碼
    r.encoding='utf8'
    soup = BeautifulSoup(r.text , "html.parser") # 把原始碼做整理
    soup_data1 = pd.read_html(str(soup.select('table')[2]),header=0)[2] ## 我不知道為什麼最後是[2]
    soup_data2 = pd.read_html(str(soup.select('table')[5]),header=0)[2]
    soup_data1=soup_data1.drop(['最高','最低','結算價','成交量','未平倉','漲跌'],axis=1)
    soup_data2=soup_data2.drop(['到期月份(週期)','最高','最低','結算價','成交量','未平倉','漲跌','履約價'],axis=1)
    df = pd.concat([soup_data1,soup_data2],axis=1).replace('-','NaN').convert_objects(convert_numeric=True)
    df.columns = ['到期月份','履約價','Call_最後成交價','Put_最後成交價']
    
    
    df['需刪除履約價'] = pd.DataFrame(df['履約價']%100)
    df['compare'] =df['Call_最後成交價'] - df['Put_最後成交價']
    
    df = df[df.需刪除履約價 != 50]
    
    df = pd.concat([df['到期月份'],df['履約價'],df['Call_最後成交價'],df['Put_最後成交價'],df['compare']],axis=1)
    df = df.reset_index(drop=True)
    
    return df

In [3]:
def  first_wen(year,month):                         # 此函數需參數 年 及 月
    day=7-(dt.date(year,month,1).weekday()+4)%7    # weekday函數 禮拜一為0;禮拜日為6
    return year,month,day

d = {}
for i in range(2010,2018):
    for j in range(1,13):
        d[i,j] = first_wen(i,j)
res = pickle.dumps(d)

B = pd.DataFrame.from_dict(d,orient='index')
B.columns = ['年','月','日']
date = pd.concat([B['年'],B['月'],B['日']],axis=1)
year = list(B['年'])
month = list(B['月'])
day =list(B['日'])

In [4]:
print(year, month, day)

[2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] [6, 3, 3, 7, 5, 2, 7, 4, 1, 6, 3, 1, 5, 2, 2, 6, 4, 1, 6, 3, 7, 5, 2, 7, 4, 1, 7, 4, 2, 6, 4, 1, 5, 3, 7, 5, 2

In [19]:
df = pd.DataFrame()

for i in range(len(year)):
    buy=GetTodayOption(year[0+i],month[0+i],day[0+i])
    #temp=temp.dropna()
    #temp = temp[np.isfinite(temp['到期月份'])]
    print(i)
    
    try:
        date=np.isnan(buy['到期月份'])
        date=list(date)
        loc=date.index(0)
        loc
        date=buy['到期月份'][loc]
        index=buy['到期月份']==date
        buy=buy[index]
        buy=buy.reset_index(drop=True)
            
            
            
        Cmp = buy['compare']
        Cmp = abs(Cmp)
        Cmp.min(skipna=True)
        loc = abs(Cmp)==Cmp.min(skipna=True)
        loc=list(loc)
        loc=loc.index(1)
        
        
        
        ######################################################################################################################
        Cmp = buy[(loc+2):(loc+3)]   ####可以改要買價外價評價內幾檔 ##########
        ######################################################################################################################
        
        
        df=pd.concat([df,Cmp])
        
    except Exception as e:
        print ('stock market was not open on %d/%d/%d' %(year[0+i], month[0+i], day[0+i]))
        pass
    

C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
stock market was not open on 2011/2/2
14
15
16
17
18
19
20
21
22
23
24
25
26
27
stock market was not open on 2012/4/4
28
29
30
31
32
33
34
35
36
37
38
39
40
stock market was not open on 2013/5/1
41
42
43
44
45
46
47
48
stock market was not open on 2014/1/1
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
stock market was not open on 2017/2/1
86
87
88
89
90
91
92
93
stock market was not open on 2017/10/4
94
95


In [20]:
df

,到期月份,履約價,Call_最後成交價,Put_最後成交價,compare
11,201001.0,8500,55.0,235.0,-180.0
7,201002.0,7700,46.0,243.0,-197.0
11,201003.0,7800,46.5,234.0,-187.5
11,201004.0,8300,35.5,210.0,-174.5
9,201005.0,7900,47.0,286.0,-239.0
7,201006.0,7300,76.0,233.0,-157.0
10,201007.0,7600,58.0,210.0,-152.0
11,201008.0,8100,32.0,218.0,-186.0
11,201009.0,7800,32.5,203.0,-170.5
12,201010.0,8500,24.0,250.0,-226.0


In [21]:
df=df.replace(201002.0,'NaN').convert_objects(convert_numeric=True).dropna()
df=df.replace(201006.0,'NaN').convert_objects(convert_numeric=True).dropna()
df=df.replace(201308.0,'NaN').convert_objects(convert_numeric=True).dropna()
df=df.replace(201502.0,'NaN').convert_objects(convert_numeric=True).dropna()
df

C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


,到期月份,履約價,Call_最後成交價,Put_最後成交價,compare
11,201001.0,8500,55.0,235.0,-180.0
11,201003.0,7800,46.5,234.0,-187.5
11,201004.0,8300,35.5,210.0,-174.5
9,201005.0,7900,47.0,286.0,-239.0
10,201007.0,7600,58.0,210.0,-152.0
11,201008.0,8100,32.0,218.0,-186.0
11,201009.0,7800,32.5,203.0,-170.5
12,201010.0,8500,24.0,250.0,-226.0
12,201011.0,8500,39.0,215.0,-176.0
11,201012.0,8700,39.5,191.0,-151.5


In [22]:
df.to_csv('buy_價外2檔.csv')